# DUCKDb

In [1]:
!pip install duckdb


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.9 MB 162.5 kB/s eta 0:01:01
   ---------------------------------------- 0.0/9.9 MB 217.9 kB/s eta 0:00:46
   ---------------------------------------- 0.1/9.9 MB 363.1 kB/s eta 0:00:28
   ---------------------------------------- 0.1/9.9 MB 476.3 kB/s eta 0:00:21
    --------------------------------------- 0.2/9.9 MB 610.0 kB/s eta 0:00:17
    --------------------------------------- 0.2/9.9 MB 620.6 kB/s eta 0:00:16
   - -------------------------------------- 0.3/9.9 MB 811.7 kB/s eta 0:00:12
   - -------------------------------------- 0.3/9.9 MB 824.9 kB/s eta 0:00:12
   - -------------------------------------- 0.3/9.9 MB 776.5 kB/s eta 0:00:13
   - -------------------------------------- 0.4/9.9 MB 802.3 kB/s eta 0:00:12
   -

In [46]:
import duckdb
import time

# Connect to DuckDB
con = duckdb.connect('my_database.duckdb')

# Load data into DuckDB
# con.execute("""
# CREATE TABLE appliances_metadata AS SELECT * FROM read_csv_auto('C:/Users/navne/Downloads/Final_Project_Phase_1/Final_Project_Phase_1/Amazon/Amazon_Appliances_Metadata.csv');
# CREATE TABLE appliances_reviews AS SELECT * FROM read_csv_auto('C:/Users/navne/Downloads/Final_Project_Phase_1/Final_Project_Phase_1/Amazon/Amazon_Appliances_Reviews.csv');
# """)

# Query: Find average price of water filters
start_time = time.time()
result_avg_price = con.execute("""

-- Average price of water filters
SELECT AVG(CAST(price AS FLOAT)) as avg_water_filter_price
FROM appliances_metadata_view
WHERE categories LIKE '%Water Filter%'
   OR title LIKE '%Water Filter%'
   OR description LIKE '%Water Filter%';

""").fetchall()
end_time = time.time()
print(f"Query execution time for average price: {end_time - start_time} seconds")
result_avg_price

 


Query execution time for average price: 0.008407115936279297 seconds


[(47.5499279821938,)]

# SQLLITE

In [25]:
import sqlite3
import csv
import time

# Connect to SQLite database (creates a new file if it doesn't exist)
conn = sqlite3.connect('amazon_appliances.db')
cursor = conn.cursor()

# Create tables
cursor.execute('''
CREATE TABLE IF NOT EXISTS appliances_metadata (
    asin TEXT PRIMARY KEY,
    price TEXT,
    imUrl TEXT,
    description TEXT,
    categories TEXT,
    title TEXT,
    brand TEXT,
    related TEXT,
    salesRank TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS appliances_reviews (
    reviewerID TEXT,
    asin TEXT,
    reviewerName TEXT,
    helpful TEXT,
    reviewText TEXT,
    overall REAL,
    summary TEXT,
    unixReviewTime INTEGER,
    reviewTime TEXT,
    PRIMARY KEY (reviewerID, asin)
)
''')

# Function to import CSV data
def import_csv(file_path, table_name):
    start_time = time.time()
    with open(file_path, 'r', encoding='utf-8') as csvfile:
        csv_reader = csv.reader(csvfile)
        headers = next(csv_reader)
        placeholders = ','.join(['?' for _ in headers])
        insert_query = f"INSERT OR REPLACE INTO {table_name} VALUES ({placeholders})"
        
        cursor.executemany(insert_query, csv_reader)
        
    conn.commit()
    end_time = time.time()
    print(f"Imported {table_name} in {end_time - start_time:.2f} seconds")

# Import data
import_csv('C:/Users/navne/Downloads/Final_Project_Phase_1/Final_Project_Phase_1/Amazon/Amazon_Appliances_Metadata.csv', 'appliances_metadata')
import_csv('C:/Users/navne/Downloads/Final_Project_Phase_1/Final_Project_Phase_1/Amazon/Amazon_Appliances_Reviews.csv', 'appliances_reviews')

# Verify data import
cursor.execute("SELECT COUNT(*) FROM appliances_metadata")
print("Metadata count:", cursor.fetchone()[0])

cursor.execute("SELECT COUNT(*) FROM appliances_reviews")
print("Reviews count:", cursor.fetchone()[0])


start_time = time.time()
# Example queries

# Average rating for water filters
cursor.execute("""
SELECT 
    COUNT(*) as total_water_filters,
    AVG(
        CASE 
            WHEN price GLOB '[0-9]*.[0-9]*' THEN CAST(price AS REAL)
            ELSE NULL
        END
    ) as avg_water_filter_price
FROM appliances_metadata
WHERE categories LIKE '%Water Filter%'
   OR title LIKE '%Water Filter%'
   OR description LIKE '%Water Filter%';

""")

result = cursor.fetchone()
end_time = time.time()

avg_price = result[1]


print(f"Average price for water filters: {avg_price:.2f}")
print(f"Query execution time: {end_time - start_time:.2f} seconds")






Imported appliances_metadata in 0.69 seconds
Imported appliances_reviews in 8.10 seconds
Metadata count: 11656
Reviews count: 143685
Average price for water filters: 44.16
Query execution time: 0.04 seconds
